## Bevezetés

Ebben a leckében szó lesz arról, hogy:
- Mi az a függvényhívás, és mire használható
- Hogyan lehet függvényhívást létrehozni az OpenAI segítségével
- Hogyan lehet egy alkalmazásba integrálni a függvényhívást

## Tanulási célok

A lecke elvégzése után tudni fogod és megérted:

- Miért érdemes függvényhívást használni
- Hogyan állítsd be a Function Call-t az OpenAI szolgáltatással
- Hogyan tervezz hatékony függvényhívásokat az alkalmazásod igényeihez


## A függvényhívások megértése

Ebben a leckében egy olyan funkciót szeretnénk fejleszteni oktatási startupunk számára, amely lehetővé teszi, hogy a felhasználók egy chatbot segítségével technikai kurzusokat találjanak. Olyan tanfolyamokat fogunk ajánlani, amelyek megfelelnek a felhasználó tudásszintjének, jelenlegi szerepkörének és az őt érdeklő technológiának.

Ehhez a következőket fogjuk használni:
 - `OpenAI` a felhasználói chat élmény megteremtéséhez
 - `Microsoft Learn Catalog API`, hogy a felhasználó igényei alapján segíthessünk kurzusokat találni
 - `Function Calling`, amellyel a felhasználó lekérdezését egy függvényhez továbbítjuk, hogy API-kérést indítsunk

Kezdésként nézzük meg, miért érdemes egyáltalán függvényhívást használni:

print("Üzenetek a következő kérésben:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # új választ kérünk a GPT-től, ahol már látja a függvény válaszát


print(second_response.choices[0].message)


### Miért hasznos a Function Calling

Ha már elvégeztél bármelyik másik leckét ebben a kurzusban, valószínűleg már érted, milyen erősek a Nagy Nyelvi Modellek (LLM-ek). Remélhetőleg az is feltűnt, hogy vannak korlátaik.

A Function Calling az OpenAI Service egyik funkciója, amely a következő problémákra kínál megoldást:

Válaszok formázásának következetlensége:
- A function calling előtt a nagy nyelvi modellek válaszai rendezetlenek és következetlenek voltak. A fejlesztőknek bonyolult ellenőrző kódokat kellett írniuk, hogy minden lehetséges válaszvariációt kezelni tudjanak.

Korlátozott integráció külső adatokkal:
- E funkció bevezetése előtt nehéz volt más alkalmazásrészekből származó adatokat beemelni a chat-környezetbe.

A válaszformátumok egységesítésével és a külső adatok egyszerű integrálásával a function calling megkönnyíti a fejlesztést, és csökkenti a plusz ellenőrző logika szükségességét.

A felhasználók például nem kaphattak olyan válaszokat, mint „Milyen most az időjárás Stockholmban?”. Ennek oka, hogy a modellek csak a tanításuk idején elérhető adatokra támaszkodtak.

Nézzük meg az alábbi példát, amely jól szemlélteti ezt a problémát:

Tegyük fel, hogy szeretnénk létrehozni egy adatbázist a diákokról, hogy a számukra legmegfelelőbb kurzust tudjuk ajánlani. Az alábbiakban két diák leírását látjuk, amelyek nagyon hasonló adatokat tartalmaznak.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Ezt el szeretnénk küldeni egy LLM-nek, hogy feldolgozza az adatokat. Később ezt felhasználhatjuk arra, hogy elküldjük egy API-nak vagy eltároljuk egy adatbázisban.

Készítsünk két teljesen azonos promptot, amelyekben utasítjuk a LLM-et, hogy milyen információkra vagyunk kíváncsiak:


Ezt el szeretnénk küldeni egy LLM-nek, hogy feldolgozza a termékünk szempontjából fontos részeket. Így két azonos promptot tudunk létrehozni, hogy utasítsuk az LLM-et:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Miután létrehoztuk ezt a két promptot, elküldjük őket az LLM-nek az `openai.ChatCompletion` használatával. A promptot a `messages` változóban tároljuk, és a szerepet `user`-re állítjuk. Ez azért van, hogy egy felhasználó üzenetét utánozzuk, amelyet egy chatbotnak írnak.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Annak ellenére, hogy a promptok ugyanazok és a leírások is hasonlóak, a `Grades` tulajdonság különböző formátumokban jelenhet meg.

Ha többször futtatod a fenti cellát, a formátum lehet `3.7` vagy `3.7 GPA` is.

Ez azért van, mert az LLM a szöveges prompt formájában kapja meg a strukturálatlan adatokat, és ugyanígy strukturálatlan adatokat is ad vissza. Szükségünk van egy strukturált formátumra, hogy tudjuk, mire számíthatunk, amikor tároljuk vagy felhasználjuk ezeket az adatokat.

A funkcióhívás használatával biztosíthatjuk, hogy strukturált adatokat kapjunk vissza. A funkcióhívás során az LLM valójában nem hív meg vagy futtat le semmilyen függvényt. Ehelyett létrehozunk egy struktúrát, amit az LLM-nek követnie kell a válaszaiban. Ezeket a strukturált válaszokat aztán arra használjuk, hogy eldöntsük, melyik függvényt futtassuk az alkalmazásainkban.


![Függvényhívási folyamatábra](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.hu.png)


### Funkcióhívások felhasználási esetei

**Külső eszközök meghívása**
A chatbotok kiválóan alkalmasak arra, hogy válaszokat adjanak a felhasználók kérdéseire. Funkcióhívások segítségével a chatbotok a felhasználók üzeneteit felhasználva bizonyos feladatokat is el tudnak végezni. Például egy diák megkérheti a chatbotot: „Küldj e-mailt az oktatómnak, hogy több segítségre van szükségem ebben a témában.” Ilyenkor a chatbot meghívhatja a `send_email(to: string, body: string)` nevű függvényt.

**API vagy adatbázis lekérdezések létrehozása**
A felhasználók természetes nyelven kereshetnek információkat, amelyeket a rendszer formázott lekérdezéssé vagy API-kéréssé alakít. Például egy tanár megkérdezheti: „Kik azok a diákok, akik teljesítették az utolsó feladatot?”, ami meghívhatja a `get_completed(student_name: string, assignment: int, current_status: string)` nevű függvényt.

**Strukturált adatok létrehozása**
A felhasználók egy szövegrészletből vagy CSV-ből a LLM segítségével kiemelhetik a legfontosabb információkat. Például egy diák egy békemegállapodásról szóló Wikipédia-cikket átalakíthat AI tanulókártyákká. Ehhez használható a `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` nevű függvény.


## 2. Első függvényhívás létrehozása

A függvényhívás létrehozása három fő lépésből áll:
1. Meghívod a Chat Completions API-t a függvényeid listájával és egy felhasználói üzenettel
2. Elolvasod a modell válaszát, hogy végrehajts egy műveletet, például egy függvény vagy API hívását
3. Újabb hívást indítasz a Chat Completions API-hoz a függvényed válaszával, hogy ezt az információt felhasználd a felhasználónak szóló válasz létrehozásához


![Egy függvényhívás folyamata](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.hu.png)


### Egy függvényhívás elemei

#### Felhasználói bemenet

Az első lépés egy felhasználói üzenet létrehozása. Ezt dinamikusan is megadhatjuk egy szövegbevitel értékének felhasználásával, vagy itt is megadhatunk egy értéket. Ha most dolgozol először a Chat Completions API-val, meg kell határoznunk az üzenet `role` (szerep) és `content` (tartalom) mezőit.

A `role` lehet `system` (szabályok létrehozása), `assistant` (a modell) vagy `user` (a végfelhasználó). Függvényhívás esetén ezt `user`-re állítjuk, és megadunk egy példakérdést.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Függvények létrehozása.

Most definiálunk egy függvényt és annak paramétereit. Itt csak egyetlen függvényt fogunk használni, amelynek neve `search_courses`, de természetesen több függvényt is létrehozhatsz.

**Fontos**: A függvények bekerülnek a rendszerüzenetbe az LLM számára, és beleszámítanak az elérhető tokenek mennyiségébe.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definíciók**

A függvénydefiníció szerkezete több szintből áll, mindegyiknek megvannak a saját tulajdonságai. Íme a beágyazott szerkezet áttekintése:

**Legfelső szintű függvénytulajdonságok:**

`name` – Annak a függvénynek a neve, amelyet meg szeretnénk hívni.

`description` – Ez írja le, hogyan működik a függvény. Itt fontos, hogy pontosak és egyértelműek legyünk.

`parameters` – Azoknak az értékeknek és formátumnak a listája, amelyeket szeretnénk, hogy a modell válaszként előállítson.

**Paraméter objektum tulajdonságai:**

`type` – A paraméter objektum adattípusa (általában "object")

`properties` – Azoknak a konkrét értékeknek a listája, amelyeket a modell a válaszához használni fog

**Egyedi paraméter tulajdonságai:**

`name` – Implicit módon a tulajdonság kulcsa határozza meg (pl. "role", "product", "level")

`type` – Ennek a konkrét paraméternek az adattípusa (pl. "string", "number", "boolean")

`description` – Az adott paraméter leírása

**Opcionális tulajdonságok:**

`required` – Egy tömb, amely felsorolja, hogy mely paraméterek szükségesek a függvényhívás végrehajtásához


### A függvény meghívása
Miután definiáltuk a függvényt, most be kell illesztenünk azt a Chat Completion API hívásába. Ezt úgy tesszük meg, hogy hozzáadjuk a `functions` paramétert a kéréshez. Ebben az esetben `functions=functions`.

Lehetőség van arra is, hogy a `function_call` értékét `auto`-ra állítsuk. Ez azt jelenti, hogy az LLM-re bízzuk, melyik függvényt hívja meg a felhasználói üzenet alapján, ahelyett, hogy mi magunk választanánk ki.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Most nézzük meg a választ, és nézzük meg, hogyan van formázva:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Láthatod, hogy a függvény neve meg van hívva, és a felhasználói üzenet alapján a LLM képes volt megtalálni az adatokat, hogy kitöltse a függvény argumentumait.


## 3. Funkcióhívások integrálása egy alkalmazásba.

Miután leteszteltük az LLM által formázott választ, most már beépíthetjük ezt az alkalmazásunkba.

### A folyamat kezelése

Ahhoz, hogy ezt beépítsük az alkalmazásunkba, kövessük az alábbi lépéseket:

Először hívjuk meg az OpenAI szolgáltatásait, és tároljuk az üzenetet egy `response_message` nevű változóban.


In [ ]:
response_message = response.choices[0].message

Most definiáljuk azt a függvényt, amely meghívja a Microsoft Learn API-t, hogy lekérje a tanfolyamok listáját:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Legjobb gyakorlatként először megnézzük, hogy a modell szeretne-e meghívni egy függvényt. Ezután létrehozunk egy elérhető függvényt, és hozzárendeljük ahhoz, amelyiket a modell hívni szeretné.

Ezután a függvény argumentumait hozzárendeljük a LLM-től kapott argumentumokhoz.

Végül hozzáfűzzük a függvényhívás üzenetét és azokat az értékeket, amelyeket a `search_courses` üzenet visszaadott. Ez minden szükséges információt megad a LLM-nek ahhoz, hogy természetes nyelven válaszoljon a felhasználónak.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kód kihívás

Szép munka! Ha szeretnéd tovább bővíteni az OpenAI Function Calling ismereteidet, próbáld ki ezt: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Adj hozzá több paramétert a függvényhez, amelyek segíthetnek a tanulóknak még több tanfolyamot találni. Az elérhető API paramétereket itt találod:  
 - Hozz létre egy másik függvényhívást, amely több információt kér be a tanulótól, például az anyanyelvét  
 - Készíts hibakezelést arra az esetre, ha a függvényhívás és/vagy az API hívás nem ad vissza megfelelő tanfolyamokat



---

**Jogi nyilatkozat**:  
Ez a dokumentum az AI-alapú [Co-op Translator](https://github.com/Azure/co-op-translator) fordítószolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti, eredeti nyelvű dokumentum tekintendő hiteles forrásnak. Kritikus információk esetén javasoljuk a professzionális, emberi fordítást. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
